In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gujarat-biomass-production/Distance_Matrix.csv
/kaggle/input/gujarat-biomass-production/preds1.csv
/kaggle/input/gujarat-biomass-production/sample_submission.csv
/kaggle/input/gujarat-biomass-production/Biomass_History.csv
/kaggle/input/gujarat-biomass-production/preds2.csv


In [82]:
distance=pd.read_csv("/kaggle/input/gujarat-biomass-production/Distance_Matrix.csv",index_col=0)
biomass=pd.read_csv('/kaggle/input/gujarat-biomass-production/Biomass_History.csv')
samplesubmission=pd.read_csv('/kaggle/input/gujarat-biomass-production/sample_submission.csv')

In [83]:
pred1=pd.read_csv('/kaggle/input/gujarat-biomass-production/preds2.csv',index_col=0)

In [84]:
pred1.sum()

2018    283809.671245
2019    326918.639318
dtype: float64

In [6]:
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '%.12f' % f
    i, p, d = s.partition('.')
    return float('.'.join([i, (d+'0'*n)[:n]]))

In [7]:
import plotly.express as px

color=pred1['2018']
fig = px.scatter_mapbox(biomass, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index"],
                        color=color,
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

- total biomass 3.84lac

In [85]:
biomass_pred=pred1
val=(pred1['2018'].values+pred1['2019'].values)/2
# val=pred1['2018'].values

In [86]:
dist=distance.values.copy()
wt=val[:,np.newaxis]+val[np.newaxis,:]
for i in range(distance.shape[0]):
    wt[i][i]=0
    dist[i][i]=1
weight=wt/dist
weight[weight==np.inf]=1e9

/tmp/ipykernel_32/252930380.py:6: RuntimeWarning:

divide by zero encountered in divide



In [87]:
%%time
class DSU:
    def __init__(self):
        self.p = {}
        self.count = 0
        self.biomass={}
        self.initvalue={}
        self.partialjoin=[]
    
    def add(self, x,val):
        if x in self.p: return
        self.p[x] = x
        self.count += 1
        self.biomass[x]=val
        self.initvalue[x]=val
    
    def find(self, x):
        if x != self.p[x]:
            self.p[x] = self.find(self.p[x])
        return self.p[x]
    
    def union(self, x, y):
        xp, yp = self.find(x), self.find(y)
        if xp == yp: return False
        self.p[xp] = yp
        self.biomass[yp]+=self.biomass[xp]
        self.count -= 1
        return True
    
    def partialunion(self,x,y,amount):
        cap=amount-self.biomass[y]
        biomassperdist=[]
        for node in range(2418):
            if self.find(node)==x:
                depo=y
                wt=1e9
                if dist[node][depo]!=0: 
                    wt=self.initvalue[node]/dist[node][depo]
                value=self.initvalue[node]
                biomassperdist.append([wt,value,node])
        biomassperdist=sorted(biomassperdist,key=lambda l:l[0], reverse=True)
        for index,node in enumerate(biomassperdist):
            if node[1]<=cap:
                self.biomass[node[2]]=self.initvalue[node[2]]
                self.biomass[y]+=node[1]
                self.p[node[2]]=y
                cap-=node[1]
            elif cap>=0.001:
                self.biomass[node[2]]=self.initvalue[node[2]]
                self.biomass[y]+=cap
                self.biomass[node[2]]-=cap
                self.initvalue[node[2]]-=cap
                biomassperdist[index][1]-=cap
                if cap>0:
                    self.partialjoin.append([cap,node[2],depo])
                cap=0
                break
        mask=[]
        value=[]
        for node in biomassperdist[index:]:
            mask.append(node[2])
            value.append(node[1])
#         print(np.sum(value))
        tempmatrix=distance.iloc[mask,mask].T.values
        tempcost=np.dot(tempmatrix,value)
        tempindex=np.argmin(tempcost)
        parent=mask[tempindex]
        self.p[parent]=parent
        self.biomass[parent]=self.initvalue[parent]
        for index,i in enumerate(mask):
            #problem parent adding its own value to make it 2 times
            if i!=parent:
                self.p[i] = parent
                self.biomass[i]=value[index]
                self.initvalue[i]=value[index]
                self.biomass[parent]+=value[index]
#         print('biomass:',parent,self.biomass[parent])
    
    def is_connected(self, x, y):
        if x in self.p and y in self.p:
            return self.find(x) == self.find(y)
        else:
            return False

CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 35 µs


In [88]:
%%time
edges=[]
for i in range(2418):
    for j in range(2418):
        if i!=j:
            edges.append([weight[i][j],i,j])
edges=sorted(edges,key=lambda l:l[0], reverse=True)
dsu=DSU()
for i in range(2418):
    dsu.add(i,val[i])
for index in range(len(edges)):
    edge=edges[index]
    value=edge[0]
    i=edge[1]
    j=edge[2]
    if dsu.biomass[i]<dsu.biomass[j] and dsu.p[i]==i and dsu.p[j]==j and dsu.biomass[j]<20000:
        if dsu.biomass[j]+dsu.biomass[i]<=20000:
            dsu.union(i,j)
        else:
            dsu.partialunion(i,j,20000)

CPU times: user 34.8 s, sys: 1.45 s, total: 36.2 s
Wall time: 35 s


In [13]:
dsu.union(2290,2323)

True

In [89]:
cluster=[]
for i in range(2418):
    cluster.append(dsu.find(i))
total=0
for j in np.unique(cluster):
    value=[]
    for i in range(2418):
        if dsu.find(i)==j:
            value.append(dsu.initvalue[i])
    for i in dsu.partialjoin:
        if dsu.find(i[2])==j:
            value.append(i[0])
    total+=np.sum(value)
    print(j,np.sum(value))
print(total)

230 19999.867506827606
249 19999.999999999993
495 19953.7296215864
606 19987.94465129191
858 20000.000000000004
1004 19996.150456801017
1291 20000.0
1341 19978.390362421094
1433 20000.0
1515 20000.0
1716 20000.0
1805 20000.0
1810 20000.000000000007
2016 19965.433428052092
2214 19999.999999999996
2222 14.12712366518073
2290 0.09533022095746245
2291 33.86242701512187
2321 5434.554373725461
305364.1552816068


In [92]:
import plotly.express as px

color=pred1['2018']
fig = px.scatter_mapbox(biomass.iloc[np.unique(cluster)], 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index"],
#                         color=color,
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [93]:
dsu.union(2290,2222)
dsu.union(2222,2291)
dsu.union(2291,2321)
dsu.union(2321,1716)
cluster=[]
for i in range(2418):
    cluster.append(dsu.find(i))
total=0
for j in np.unique(cluster):
    value=[]
    for i in range(2418):
        if dsu.find(i)==j:
            value.append(dsu.initvalue[i])
    for i in dsu.partialjoin:
        if dsu.find(i[2])==j:
            value.append(i[0])
    total+=np.sum(value)
    print(j,np.sum(value))
print(total)

230 19999.867506827606
249 19999.999999999993
495 19953.7296215864
606 19987.94465129191
858 20000.000000000004
1004 19996.150456801017
1291 20000.0
1341 19978.390362421094
1433 20000.0
1515 20000.0
1716 25482.639254626716
1805 20000.0
1810 20000.000000000007
2016 19965.433428052092
2214 19999.999999999996
305364.1552816068


In [94]:
uniquecluster=np.unique(cluster)
noofcluster=len(uniquecluster)
clustermask=[]
clustermap={}
for i in range(noofcluster):
    clustermap[uniquecluster[i]]=i
    clustermask.append([])
for i in range(2418):
    value=clustermap[cluster[i]]
    clustermask[value].append(i)
    

depoloc=[]
for index,mask in enumerate(clustermask):
    tempmatrix=distance.iloc[mask,mask].T.values
    tempvalue=val[mask]
    tempcost=np.dot(tempmatrix, tempvalue)
    partialedge=[]
    for i in dsu.partialjoin:
        if dsu.find(i[2])==uniquecluster[index]:
            partialedge.append([i[0],i[1]])
    for index,i in enumerate(mask):
        cost=0
        for j in partialedge:
            cost+=(j[0]*dist[j[1]][i])
        tempcost[index]+=cost
    tempindex=np.argmin(tempcost)
    depoloc.append(mask[tempindex])

In [16]:
# oldtonewdepomap={}
# newtoolddepomap={}
# for i in range(noofcluster):
#     old=uniquecluster[i]
#     new=depoloc[i]
#     oldtonewdepomap[old]=new
#     newtoolddepomap[new]=old

# depomap={}
# for index,depo in enumerate(depoloc):
#     depomap[depo]=index

In [99]:
biomassperdist=[]
for node in range(2418):
    stats=[]
    for depo in depoloc:
        wt=1e9
        if dist[node][depo]!=0: 
            wt=val[node]/dist[node][depo]
        value=val[node]
        stats.append([wt,value,node,depo])
    stats=sorted(stats,key=lambda l:l[0], reverse=True)
    biomassperdist.append(stats)
biomassperdist=sorted(biomassperdist,key=lambda l:l[0][0], reverse=True)

In [100]:
knapsack={}
capacity={}
for depo in depoloc:
    capacity[depo]=20000
    knapsack[depo]=[0]*2418
index=0
while index<2418:
    stats=biomassperdist[index]
    node=stats[0]
    depo=node[3]
    if capacity[depo]>=node[1]:
        knapsack[depo][node[2]]=node[1]
        capacity[depo]-=node[1]
        index+=1
    else:
        totalcap=0
        for tempdepo in depoloc:
                totalcap+=capacity[tempdepo]
        if totalcap<0.001:
                break
        red=capacity[depo]
        knapsack[depo][node[2]]=capacity[depo]
        capacity[depo]=0
        del stats[0]
        if(len(stats)==0):
            index+=1
            continue
        for i in range(len(stats)):
            stats[i][1]=(stats[i][1]-red)
            stats[i][0]-=(red/dist[stats[i][2]][stats[i][3]])
        biomassperdist[index]=sorted(stats,key=lambda l:l[0], reverse=True)
        biomassperdist=biomassperdist[:index]+sorted(biomassperdist[index:],key=lambda l:l[0][0], reverse=True)

In [19]:
# total=[]
# for depo in depoloc:
#     total.append(knapsack[depo])
# cap=np.sum(total,axis=1)

In [101]:
depovalue=[]
for depo in depoloc:
    mask=knapsack[depo]
    cost=0
    for node,value in enumerate(mask):
        if(value>0):
            value=truncate(value,3)
            cost+=value
    depovalue.append(cost)
depoloc,depovalue

([309,
  292,
  443,
  547,
  985,
  1060,
  1232,
  1405,
  1432,
  1454,
  1828,
  1860,
  1811,
  1845,
  2215],
 [19999.880000000012,
  19999.920999999988,
  19999.945,
  19999.934999999998,
  19999.967999999993,
  19999.917999999983,
  19999.959000000003,
  19999.950999999997,
  19999.934999999998,
  19999.947999999997,
  19999.952999999987,
  19999.978,
  19999.961,
  19999.931,
  19999.89000000001])

In [ ]:
'''import plotly.express as px

fig = px.scatter_mapbox(biomass.iloc[samplesubmission[samplesubmission['data_type']=='depot_location'].source_index.values], 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index", "2010"],
#                         color='black',
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()'''

In [102]:
import plotly.express as px

fig = px.scatter_mapbox(biomass.iloc[depoloc], 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index", "2010"],
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [103]:
depoprod={}
for index,depo in enumerate(depoloc):
    depoprod[depo]=depovalue[index]

In [87]:
# hardcode refinery clusters
refmask=[[1037,1324,1687,1907,1746],
        [988,1641,2030,1813,1470],
        [148,865,547,687,442],
        [1283,1291,1488,1937,2102]]

In [40]:
# hardcode refinery clusters for part2 with correct clusters using partialunion 
refmask=[[2321,1718,1432,1232],
        [2172,1860,1905],
        [102,609,443,881],
        [985,1325,1333,1402,1531]]

In [95]:
refmask=[[687,882,1174,1548,2049],
        [1456,1849,1472,1860,2129],
        [648,1091,929,346,102]]

In [23]:
refmask=[[754,1120,1553,1718,2264],
        [1469,1637,1849,1861,2172],
        [102,438,609,985,1142]]

In [104]:
refmask=[[292,309,443,547,985],
        [1454,1845,1811,1860,2215],
        [1060,1405,1232,1432,1828]]

In [24]:
# oldtonewdepomap={}
# newtoolddepomap={}
# for i in range(noofcluster):
#     old=uniquecluster[i]
#     new=depoloc[i]
#     oldtonewdepomap[old]=new
#     newtoolddepomap[new]=old

# depomap={}
# for index,depo in enumerate(depoloc):
#     depomap[depo]=index

In [105]:
def calcdepoloc(depoloc,refmask):
    depolocimprove=[]
    refmaskimprove=[]
    
    depotorefmap={}
    for index in range(len(refmask)):
        mask=refmask[index]
        refmaskimprove.append([])
        for i in mask:
            depotorefmap[i]=index
            
    depoprod={}
    for index,depo in enumerate(depoloc):
        depoprod[depo]=depovalue[index] 
    
    for index,mask in enumerate(clustermask):
        depono=depoloc[index]
        refindex=depotorefmap[depono]
        refcluster=refmask[refindex]
        tempmatrix=distance.iloc[mask,mask].T.values
        tempvalue=val[mask]
        tempcost=np.dot(tempmatrix, tempvalue)
        partialedge=[]
        for i in dsu.partialjoin:
            if dsu.find(i[2])==uniquecluster[index]:
                partialedge.append([i[0],i[1]])
        for index,i in enumerate(mask):
            cost=0
            for j in partialedge:
                cost+=(j[0]*dist[j[1]][i])
            tempcost[index]+=cost
        for index,i in enumerate(mask):
            cost=0
            for neardepo in refcluster:
                cost+=depoprod[neardepo]*dist[neardepo][i]
            tempcost[index]+=cost
        tempindex=np.argmin(tempcost)
        minicost=np.min(tempcost)
        depolocimprove.append(mask[tempindex])
        
    for index,depo in enumerate(depoloc):
        refindex=depotorefmap[depo]
        deponew=depolocimprove[index]
        refmaskimprove[refindex].append(deponew)
        
    return depolocimprove,refmaskimprove,minicost

In [106]:
depolocimprove=depoloc
refmaskimprove=refmask
for i in range(10):
    depolocimprove,refmaskimprove,minicost=calcdepoloc(depolocimprove,refmaskimprove)
    print(minicost,depolocimprove)

11696482.10330165 [219, 292, 346, 488, 864, 1226, 1232, 1221, 1361, 1579, 1547, 1860, 1809, 1855, 2026]
7052506.5480570495 [219, 215, 437, 488, 864, 1226, 1232, 1221, 1361, 1801, 1486, 1858, 1861, 1855, 2026]
5561121.9136605505 [219, 215, 437, 488, 864, 1226, 1232, 1221, 1361, 1801, 1486, 1858, 1861, 1855, 2026]
5561121.9136605505 [219, 215, 437, 488, 864, 1226, 1232, 1221, 1361, 1801, 1486, 1858, 1861, 1855, 2026]
5561121.9136605505 [219, 215, 437, 488, 864, 1226, 1232, 1221, 1361, 1801, 1486, 1858, 1861, 1855, 2026]
5561121.9136605505 [219, 215, 437, 488, 864, 1226, 1232, 1221, 1361, 1801, 1486, 1858, 1861, 1855, 2026]
5561121.9136605505 [219, 215, 437, 488, 864, 1226, 1232, 1221, 1361, 1801, 1486, 1858, 1861, 1855, 2026]
5561121.9136605505 [219, 215, 437, 488, 864, 1226, 1232, 1221, 1361, 1801, 1486, 1858, 1861, 1855, 2026]
5561121.9136605505 [219, 215, 437, 488, 864, 1226, 1232, 1221, 1361, 1801, 1486, 1858, 1861, 1855, 2026]
5561121.9136605505 [219, 215, 437, 488, 864, 1226, 1232,

In [107]:
depoprod={}
for index,depo in enumerate(depolocimprove):
        depoprod[depo]=depovalue[index] 

In [108]:
refloc=[]
minicost=0
for index,mask in enumerate(refmaskimprove):
    tempmatrix=distance.iloc[mask,:].T.values
    tempvalue=[]
    for i in mask:
        tempvalue.append(depoprod[i])
    tempcost=np.dot(tempmatrix, tempvalue)
    tempindex=np.argmin(tempcost)
    minicost+=np.min(tempcost)
    
    refloc.append(tempindex)
print(minicost)

14224046.455044799


In [109]:
refloc

[488, 1858, 1232]

In [90]:
'''%%time
mini=1e18
refinerylocation=[]
temprefmask=[[],[],[],[]]
refmask=[]
itercount=0
def solve(index,depoloc):
    global mini,refinerylocation,temprefmask,refmask,itercount
    if index>=len(depoloc):
        refloc=[]
        minicost=0
        for index,mask in enumerate(temprefmask):
            tempmatrix=distance.iloc[mask,:].T.values
            tempvalue=[]
            for i in mask:
                tempvalue.append(depoprod[i])
            tempcost=np.dot(tempmatrix, tempvalue)
            tempindex=np.argmin(tempcost)
            minicost+=np.min(tempcost)
            refloc.append(tempindex)
        if(minicost<mini):
            mini=minicost
            refinerylocation=refloc
            refmask=temprefmask
        if(itercount%10000==0):
            print(itercount,mini,refinerylocation)
        itercount+=1
        return
    
    for i in range(4):
        if len(temprefmask[i])<5:
            temprefmask[i].append(depoloc[index])
            solve(index+1,depoloc)
            temprefmask[i].remove(depoloc[index])
    return

solve(0,depoloc)'''

0 49263703.15581255 [547, 988, 1583, 1813]
10000 33867793.11742823 [547, 1148, 1424, 1746]
20000 33867793.11742823 [547, 1148, 1424, 1746]
30000 33867793.11742823 [547, 1148, 1424, 1746]
40000 33867793.11742823 [547, 1148, 1424, 1746]
50000 33867793.11742823 [547, 1148, 1424, 1746]
60000 33867793.11742823 [547, 1148, 1424, 1746]
70000 33867793.11742823 [547, 1148, 1424, 1746]
80000 33867793.11742823 [547, 1148, 1424, 1746]
90000 33867793.11742823 [547, 1148, 1424, 1746]
100000 33867793.11742823 [547, 1148, 1424, 1746]
110000 29809781.11131997 [547, 1359, 1148, 1746]
120000 29809781.11131997 [547, 1359, 1148, 1746]
130000 29809781.11131997 [547, 1359, 1148, 1746]
140000 29809781.11131997 [547, 1359, 1148, 1746]
150000 29809781.11131997 [547, 1359, 1148, 1746]
160000 29809781.11131997 [547, 1359, 1148, 1746]
170000 29809781.11131997 [547, 1359, 1148, 1746]
180000 29809781.11131997 [547, 1359, 1148, 1746]
190000 29809781.11131997 [547, 1359, 1148, 1746]


KeyboardInterrupt: 

In [92]:
refloc=refinerylocation
refloc

[547, 1359, 1148, 1746]

In [110]:
def knapsackmask(val,dist,depoloc):
        biomassperdist=[]
        for node in range(2418):
            stats=[]
            for depo in depoloc:
                wt=1e9
                if dist[node][depo]!=0: 
                    wt=val[node]/dist[node][depo]
                value=val[node]
                stats.append([wt,value,node,depo])
            stats=sorted(stats,key=lambda l:l[0], reverse=True)
            biomassperdist.append(stats)
        biomassperdist=sorted(biomassperdist,key=lambda l:l[0][0], reverse=True)
        knapsack={}
        capacity={}
        for depo in depoloc:
            capacity[depo]=20000
            knapsack[depo]=[0]*2418
        index=0
        while index<2418:
            stats=biomassperdist[index]
            node=stats[0]
            depo=node[3]
            if capacity[depo]>=node[1]:
                knapsack[depo][node[2]]=node[1]
                capacity[depo]-=node[1]
                index+=1
            else:
                totalcap=0
                for tempdepo in depoloc:
                    totalcap+=capacity[tempdepo]
                if totalcap<0.001:
                    break
                red=capacity[depo]
                knapsack[depo][node[2]]=capacity[depo]
                capacity[depo]=0
                del stats[0]
                if(len(stats)==0):
                    index+=1
                    continue
                for i in range(len(stats)):
                    stats[i][1]=(stats[i][1]-red)
                    stats[i][0]-=(red/dist[stats[i][2]][stats[i][3]])
                biomassperdist[index]=sorted(stats,key=lambda l:l[0], reverse=True)
                biomassperdist=biomassperdist[:index]+sorted(biomassperdist[index:],key=lambda l:l[0][0], reverse=True)
        return knapsack

In [111]:
knapsack18=knapsackmask(pred1['2018'].values,dist,depolocimprove)
knapsack19=knapsackmask(pred1['2019'].values,dist,depolocimprove)

In [131]:
total=[]
for depo in depolocimprove:
    total.append(knapsack18[depo])
(np.sum(total,axis=0)<=pred1['2018'].values).sum()

2417

In [132]:
pred1[np.sum(total,axis=0)>pred1['2018'].values]

,2018,2019
1146,177.747016,544.067923


In [133]:
np.sum(total,axis=0)[1146]

177.74701602277207

In [135]:
depoloc=depolocimprove
refmask=refmaskimprove
depomap={}
for index,depo in enumerate(depoloc):
    depomap[depo]=index

In [136]:
location=[]
for i in depoloc:
    location.append([20182019,'depot_location',i,np.nan,np.nan])
for i in refloc:
    location.append([20182019,'refinery_location',i,np.nan,np.nan])
loc_df=pd.DataFrame(location,columns=samplesubmission.columns)

In [137]:
biomass_forcast=samplesubmission[samplesubmission['data_type']=='biomass_forecast']
pred=np.concatenate((pred1['2018'],pred1['2019']))
biomass_forcast['value']=pred
biomass_forcast=biomass_forcast.reset_index().drop(columns=['index'])

/tmp/ipykernel_32/2742677146.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [119]:
2418+1884

4302

In [134]:
biomass_forcast.loc[1146]

year                             2018
data_type            biomass_forecast
source_index                     1146
destination_index                 NaN
value                      177.747016
Name: 1146, dtype: object

In [138]:
biomass_forcast.loc[4302,'value']=188.152
biomass_forcast.loc[1146,'value']=177.748

In [139]:
transport=[]
# depo
depovalue=[[],[]]
#2018
for depo in depoloc:
    mask=knapsack18[depo]
    cost=0
    for node,value in enumerate(mask):
        if(value>0):
#             value=int(value)
            value=truncate(value,3)
            transport.append([2018,'biomass_demand_supply',node,depo,value])
            cost+=value
    depovalue[0].append(cost)
#2019
for depo in depoloc:
    mask=knapsack19[depo]
    cost=0
    for node,value in enumerate(mask):
        if(value>0):
#             value=int(value)
            value=truncate(value,3)
            transport.append([2019,'biomass_demand_supply',node,depo,value])
            cost+=value
    depovalue[1].append(cost)

#ref
#2018
for index,mask in enumerate(refmask):
    for i in mask:
        value=depovalue[0][depomap[i]]
        source=i
        destination=refloc[index]
        transport.append([2018,'pellet_demand_supply',source,destination,value])
#2019
for index,mask in enumerate(refmask):
    for i in mask:
        value=depovalue[1][depomap[i]]
#         print(value)
        source=i
        destination=refloc[index]
        transport.append([2019,'pellet_demand_supply',source,destination,value])
transport_df=pd.DataFrame(transport,columns=samplesubmission.columns)

loc_df+biomass_forcast+transport_df

In [140]:
submission=pd.concat([loc_df,biomass_forcast,transport_df],axis=0)

In [141]:
submission.to_csv('submission_with_pred2_withrefdist_improved_avgofyear.csv',index=False)